# Regression Model A: Field 5

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 5 (F5)
Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E5" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E5" ),]
dE4=d4[which(d4$Estate=="E5" ),]
dE5=d5[which(d5$Estate=="E5" ),]

## Variables

Declare the variable and construct new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL
foliar=d2$Diff
prun.time=dE3$Frequency
prun.cost=dE3$PruningCost
manuring.time=dE4$Frequency
manuring.cost=dE4$ManuringCost
pd.time=dE5$Frequency
pd.cost=dE5$PDCost

dg1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Correlation

Check the correlations for all variables. 

In [6]:
c1=cor(dg1[,-1:-2])

corrplot(c1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(c1, type="upper")

## Train and test data

Split the dataset into train and test sets. 

After a model has been processed by using the training set, we test the model by making predictions against the test set. 

Because the data in the testing set already contains known values for the attribute that we want to predict, it is easy to determine whether the model's guesses are correct.

We use the datasets from May 2014 until December 2016 is considered for training dataset. The test dataset is from January 2017 until December 2017. 

In [7]:
train = dg1[dg1$year<=2016,]
test = dg1[dg1$year==2017,]
dim(train)
dim(test)

[1] 32 11

[1] 12 11

## Regression

Fit the data using multiple regression for all variables. 

In [8]:
a1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=train)
summary(a1)
plot(a1,which=1)
plot(a1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-64.156 -11.936   0.914  22.722  39.402 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.596e+02  2.326e+01   6.861 5.38e-07 ***
rainfall       3.868e-02  5.712e-02   0.677   0.5050    
foliar        -4.308e+00  1.957e+00  -2.202   0.0380 *  
prun.time     -4.385e+00  6.594e+00  -0.665   0.5126    
prun.cost      5.273e-03  9.274e-03   0.569   0.5752    
manuring.time  2.867e+00  7.400e+00   0.387   0.7020    
manuring.cost  6.339e-06  3.278e-04   0.019   0.9847    
pd.time       -1.787e+01  9.212e+00  -1.940   0.0648 .  
pd.cost        4.162e-03  6.237e-03   0.667   0.5112    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 29.82 on 23 degrees of freedom
Multiple R-squared:  0.401,	Adjusted R-squared:  0.1927

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 


The stepwise method incorporate with VIF factor is better than anova method in order to     select the significant variables.    

### Using ANOVA

In [9]:
anova(a1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,133.07828,133.07828,0.14963542,0.702439916
foliar,1,9526.32196,9526.32196,10.71155381,0.003341409
prun.time,1,55.66808,55.66808,0.06259411,0.804663082
prun.cost,1,39.69559,39.69559,0.04463438,0.834538524
manuring.time,1,154.31554,154.31554,0.17351494,0.680868462
manuring.cost,1,82.80559,82.80559,0.09310797,0.763006863
pd.time,1,3306.01188,3306.01188,3.71733438,0.066285903
pd.cost,1,396.03154,396.03154,0.44530440,0.511213617
Residuals,23,20455.05338,889.35015,NA,NA


### Using stepwise

In [10]:
selectedMod <- step(a1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=224.73
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- manuring.cost  1       0.3 20455 222.73
- manuring.time  1     133.5 20589 222.94
- prun.cost      1     287.5 20743 223.18
- prun.time      1     393.4 20848 223.34
- pd.cost        1     396.0 20851 223.34
- rainfall       1     407.9 20863 223.36
<none>                       20455 224.73
- pd.time        1    3346.1 23801 227.58
- foliar         1    4311.3 24766 228.85

Step:  AIC=222.73
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- manuring.time  1     250.0 20705 221.12
- prun.cost      1     290.3 20746 221.18
- pd.cost        1     397.9 20853 221.34
- rainfall       1     417.9 20873 221.38
- prun.time      1     423.5 20879 221.38
<none>                       20455 222.73
- pd.time        1    3429.8 23885 225.69
- foli


Call:
lm(formula = yield ~ foliar + pd.time, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-63.514 -10.322  -0.629  21.681  41.546 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  167.693     13.720  12.222 5.79e-13 ***
foliar        -4.610      1.476  -3.123  0.00404 ** 
pd.time      -13.884      6.627  -2.095  0.04500 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 27.31 on 29 degrees of freedom
Multiple R-squared:  0.3664,	Adjusted R-squared:  0.3227 
F-statistic: 8.386 on 2 and 29 DF,  p-value: 0.001337


  foliar  pd.time 
1.028261 1.028261 



Call:
lm(formula = yield ~ foliar + pd.time, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-63.514 -10.322  -0.629  21.681  41.546 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  167.693     13.720  12.222 5.79e-13 ***
foliar        -4.610      1.476  -3.123  0.00404 ** 
pd.time      -13.884      6.627  -2.095  0.04500 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 27.31 on 29 degrees of freedom
Multiple R-squared:  0.3664,	Adjusted R-squared:  0.3227 
F-statistic: 8.386 on 2 and 29 DF,  p-value: 0.001337


## Prediction

The performance of the model is evaluated by comparing the estimated value with the actual value. In this case, the test dataset (2017 datasets) is used to examine the performance of the model with significant variables. 

Construct the table and graph that represents the estimated and observed value. 

In [11]:
estimate=predict(selectedMod,newdata=test)
compare=data.frame(test$yield,estimate)
names(compare)=c("Actual","Estimate")
compare

plot(compare$Actual,type="b",col="blue",xlab="Actual",ylab="Predicted",pch=19)
lines(fitted(selectedMod),col="red",type="b",pch=18)
legend("topleft",legend=c("Actual","Predicted"),col=c("blue","red"),lty=1,cex=0.8)

,Actual,Estimate
,<dbl>,<dbl>
33,137.894,89.15879
34,90.749,98.43276
35,63.475,79.93835
36,64.678,116.98071
37,83.421,167.69313
38,103.095,163.08291
39,140.657,130.70431
40,145.893,153.86247
41,160.711,149.25225


## Root mean square error (RMSE)

Calculate the root mean square error (RMSE) of the model. Extract the coefficient of determination (R-square) to know how close the data to the fitted regression line. 

In [12]:
rmse=rmse(test$yield,estimate )
fit=c(RMSE = rmse, R2=summary(a1)$r.squared)
fit

RMSE         R2 
43.8114777  0.4010055